# フィードの収集

英語フィードの収集
- 1_1_feeds.py と同じ内容

feedparserパッケージが無いとエラーが出る場合は、端末（ターミナル等）で以下のコマンドによりインストール：
```
conda install feedparser
```

In [50]:
import pandas as pd
import feedparser
import re
import requests
from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text
import io

def get_webpage_content(url):
    # URLからデータ取得 (GET)
    print('URL # {}'.format(url), flush=True)
    res = requests.get(url)
    content_type = res.headers['Content-Type']
    print('content_type # {}'.format(content_type), flush=True)
    if 'text/html' in content_type:
        soup = BeautifulSoup(res.content, 'html.parser')
        main_txt = soup.main
        # main_text == Noneならmainタグがない
        if main_txt == None:
            text = soup.get_text()
            text.replace('\n', '').replace('\u3000', ' ')
        else:
            text = soup.main.get_text()
            text.replace('\n', '').replace('\u3000', ' ')
    elif 'application/pdf':
        text2 = []
        ext_text = extract_text(io.BytesIO(res.content))
        
        # 行ごとに分割してから不要な改行コードを削除
        # extract_text.replace('\n', '')
        for l in ext_text.split('\n\n'):
            text2.append(l.replace('\n', ''))
        # その後もう一度結合し直す
        text = ' '.join(text2)
    return text

In [51]:
def get_feeds(n, url):
    """
    urlから取得したフィードをリストにして返す。
    """
    # print('URL #{} {}'.format(n, url), flush=True)
    # 初期化
    feeds = []
    # urlからフィードを取得
    try:
        f = feedparser.parse(url)
        if f.bozo != 0 and f.bozo != True:
            print('Error(bozo) url:', url, flush=True)
            return feeds
    except:
        print('Error(exception) url:', url, flush=True)
        return feeds
    # f.entries 内の各要素について処理
    # - title: タイトル
    # - summary, description: 内容
    for e in f.entries:
        # タイトル
        if 'title' in e:
            title = e.title
        else:
            title = ''

        # リンク(コンテンツ本体のURL)
        if 'link' in e:
            link = e.link
        else:
            link = ''

        # 内容：summary または description
        if 'summary' in e:
            body = e.summary
        elif 'description' in e:
            body = e.description
        else:
            body = ''
        
        # title と body の両方が空ならば追加しない
        if title == '' and body == '':
            continue

        # HTML 形式の場合があるため <...> を削除
        body = re.compile(r'<[^>]+>').sub('', body)
        # feeds に URL, タイトル、内容 を追加
        # - body.strip(): 先頭、末尾の改行・空白文字を削除

        content = get_webpage_content(link)

        feeds.append([url, title, link, body.strip(), content])

    return feeds


In [57]:
def write_feeds(feedlist, output, simple_output):
    """
    feedlistに記載のURLからフィードを取得し、CSV形式でoutputファイルに書き出す。
    outputファイルが既にあれば、読み込み、重複排除を行う。
    """
    try:
        # outputファイル（CSV形式）から読み込み
        df = pd.read_csv(output)
        s_df = pd.read_csv(simple_output)
    except:
        # outputファイルがなかった場合、DataFrameを作成
        df = pd.DataFrame([], columns=['url', 'title', 'link', 'summary', 'content'])
        s_df = pd.DataFrame([], columns=['title', 'link'])

    # feedlistに記載のURLからフィードを取得
    urls = [line.strip() for line in open(feedlist)]
    for i, url in enumerate(urls):
        feeds = get_feeds(i, url)
        feeds_df = pd.DataFrame(feeds, columns=['url', 'title', 'link', 'summary', 'content'])
        df = pd.concat([df, feeds_df])
        s_df = pd.concat([s_df, feeds_df.loc[:, ['title', 'link']]])

    # 重複排除
    df = df.drop_duplicates()
    s_df = s_df.drop_duplicates()
    # CSV形式でoutputファイルに書き出し
    df.to_csv(output, index=False)
    s_df.to_csv(simple_output, index=False)



In [58]:
# フィードの取得、書き出し
# ハッカーニュースのRSSの場合、その時点での人気の記事が順番に表示されている
write_feeds('feedlist_en.txt', 'output_en.csv', 'index_output.csv')
# write_feeds('feedlist_jp.txt', 'output_jp.csv')

URL # https://daverupert.com/2023/01/shadow-banned-by-duckduckgo-and-bing/
content_type # text/html; charset=UTF-8
URL # https://github.com/Enerccio/SLT
content_type # text/html; charset=utf-8
URL # https://news.ycombinator.com/item?id=34388866
content_type # text/html; charset=utf-8
URL # https://utcc.utoronto.ca/~cks/space/blog/linux/Ubuntu2204ServerPhasedUpdates
content_type # text/html; charset=UTF-8
URL # https://dbohdan.com/scripts-with-dependencies
content_type # text/html; charset=utf-8
URL # https://blog.revolutionanalytics.com/2014/01/the-fourier-transform-explained-in-one-sentence.html
content_type # text/html; charset=utf-8
URL # https://gofoss.net/
content_type # text/html; charset=UTF-8
URL # https://github.com/andrewcmyers/constrain
content_type # text/html; charset=utf-8
URL # https://jobs.flightcontrol.dev/developer-advocate
content_type # text/html; charset=utf-8
URL # https://gitlab.com/tsoding/porth
content_type # text/html; charset=utf-8
URL # https://old.reddit.co

In [48]:
link = 'https://www.sqlite.org/fasterthanfs.html'
get_webpage_content(link)

URL # https://www.sqlite.org/fasterthanfs.html
content_type # text/html; charset=utf-8


'\n\n\n\n\n35% Faster Than The Filesystem\n\n\n\n\n\n\n\n\n\nSmall. Fast. Reliable.Choose any three.\n\n\n\nHome\nMenu\nAbout\nDocumentation\nDownload\nLicense\nSupport\nPurchase\n\nSearch\n\n\n\n\nAbout\nDocumentation\nDownload\nSupport\nPurchase\n\n\n\n\n\nSearch Documentation\nSearch Changelog\n\n\n\n\n\n\n\n\n\n\n35% Faster Than The Filesystem\n\n\n\n►\nTable Of Contents\n\n1. Summary\n1.1. Caveats\n1.2. Related Studies\n2. How These Measurements Are Made\n2.1. Read Performance Measurements\n2.2. Write Performance Measurements\n2.3. Variations\n3. General Findings\n4. Additional Notes\n4.1. Compiling And Testing on Android\n\n\n\n\n1. Summary\nSQLite reads and writes small blobs (for example, thumbnail images)\n35% faster¹ than the same blobs\ncan be read from or written to individual files on disk using\nfread() or fwrite().\n\nFurthermore, a single SQLite database holding\n10-kilobyte blobs uses about 20% less disk space than\nstoring the blobs in individual files.\n\nThe perform